## Atoms

AtomBuilder3 (AB3) builds charge field atomic models selected by 
the user. AB3 is a copy of mBuilder, utilizing a datafile, and one 
and two directional proton configuration slot groups replacing earlier 
versions, AtomBuilder (AB) and AtomBuilder2 (AB2). AB and AB2 are now 
in the additional notebooks folder as history files. mbuilder carries 
on as the day-to-day working program intended to develop a 3D charge 
field model rendering of molecular matter selected by the user.  

Operating instructions. Start by clicking on the "Atom data" cell,
about 10 cells below. Next, from the toolbar's Cell dropdown selection 
menu choose "Run All Above". When the gui in the cell above "Atom data" 
becomes visible, make any changes to it or select an atom from the 
Periodic table. Next, with the gui or the Periodic table cell still
active, select "Run All Below" from the Cell dropdown menu. You'll then 
view the main program outputs from the tab enclosure cell near the 
bottom. For any changes, repeat - go back to the gui or Periodic 
table, make changes, then select "Run All Below". These instructions 
are repeated in the "Atom data" cell.

Atoms is intended to introduce Charge field theory - see the 
notebook's end cell's source references.                                          

In [ ]:
from pythreejs import *
from IPython.display import display
from ipywidgets import HBox, VBox, Button, ButtonStyle, GridspecLayout
import ipywidgets as widgets
from math import sqrt, tan, copysign
from random import random, randint, seed
import pandas as pd
import numpy as np

In [ ]:
# constants

PHI = (1 + sqrt(5)) / 2  # calculating phi-based-pi, PI_PHI.
PI_PHI = 4 / sqrt(PHI)
PROTON_RADIUS = 0.05  # proton radius, used by all atomic distances
CAROUSEL_ROTATION_SECONDS = (
    5.0  # Problem Here # time in secs for the carousel to make one rotation
)

In [ ]:
# pythreejs config

VIEW_WIDTH = 600
VIEW_HEIGHT = 400

camera_z_axis_horiz = CombinedCamera(
    position=[0, 0, 10], up=[0, 1, 0], width=VIEW_WIDTH, height=VIEW_HEIGHT
)

camera_z_axis_vert = CombinedCamera(
    position=[0, 0, 10], up=[0, 0, 1], width=VIEW_WIDTH, height=VIEW_HEIGHT
)

key_light = DirectionalLight(position=[0, 5, 5])
ambient_light = AmbientLight()

In [ ]:
# Read element data

FILENAME = "data/Cr6-Elements-M.csv"
df = pd.read_csv(
    FILENAME,
    dtype={
        "PX": float,
        "PY": float,
        "PZ": float,
        "P2P3": float,
        "P12": float,
        "PE": float,
    },
    true_values=["Y"],
    false_values=["N"],
)
df = df.fillna(" ")

df["AtomicSlotgroup"] = df["AtomicNumber"].copy()
df["AtomicNumber"] = df["AtomicNumber"].astype(str)

df.head()

## Elements and Molecules

Elements is a list of tuples used to  
identify or re-select a desired atom from the gui 
dropdown or periodic table. Also used as the optional
3D atom's atomic label. elements is taken from the 
df['AtomicTuple'] column.  

Starting with 1710 rows. After drop_duplicates, the 
'AtomicTuple' row index increments by 19's, 0 to 1709, 
19x expanded. The Series.array method renumbers the 
index back to the desired 0-89. The two steps are 
chained.

Next, atomictuples has been read as a list of strings. 
Those strings are converted into tuples by using 
the map and eval methods.



In [ ]:
atomic_tuples = df["AtomicTuple"].drop_duplicates().array
elements = list(map(eval, atomic_tuples))

molecules = {"H2": 1, "C2": 2, "N2": 3, "O2": 4}

# (total number of atoms, first atom's atomic number, second atom's atomic number)
molecule_config = [(2, 1, 1), (2, 6, 6), (2, 7, 7), (2, 8, 8)]

Click to select an element from the periodic table

Most of the controls will be displayed in the cell following the periodic table. The element can also be selected by a dropdown widget.

In [ ]:
from helpers.render import render_periodic_table, render_control_panel

In [ ]:
# display periodic table
grid, color_list, a_buttons = render_periodic_table(df)
for i in range(len(a_buttons)):

    def on_clicked(b):
        selectAtom.value = i + 1

    a_buttons[i].on_click(on_clicked)

grid

In [ ]:
# display the control panel
(
    guiwidgets,
    selected_atomic_number,
    display_neutrons,
    colored_emissions,
    emission_type,
    emission_radius,
    s1s2,
    p2p3Dist,
    neutron_orbit_radius,
    c_angle_degree,
    camera_mode,
    camera_orient,
    camera_type,
    atoms_label,
    selectAtom
) = render_control_panel(elements)
guiwidgets

In [ ]:
# Selected Atom
atomic_symbols = df["AtomicSymbol"].drop_duplicates().array
An = elements[selected_atomic_number - 1]
atomic_label = An[0]
atomic_number = selected_atomic_number
element_list = df["Element"].drop_duplicates().array
atom = element_list[selected_atomic_number - 1]
atomsymbol = atomic_symbols[selected_atomic_number - 1]
rowstart = 19 * (atomic_number - 1)
rowstop = rowstart + 18  # Used with the loc slicing
slotsetA_df = df.loc[rowstart:rowstop, "SlotNumber":"AtomicSlotgroup"].set_index(
    "SlotNumber"
)
slotlistA = list(slotsetA_df.loc[:, "Protons"])

print(slotlistA)
slotsetA_df.head()

Create the Slotlayout Diagram.

The slotlist, a set of 19 integers identifying
the number of protons in each numbered slot is
found in the df['Protons'] column.

Using 45 non-interactive buttons to create the
atom's slotlayout (19 + 19 + 7 + 1) diagram: 19
in the center SL configuration; 19 for the left
(1,3,5,...,19) and right(2,4,6,...,18) columns;
7 for the slot/proton count color legend at the
bottom; and 1 atomic symbol/mumber at the bottom
right. All buttons except the bottom right incl:
proton count color, layout width and layout height.

The 7 colors used for color-coding each slot's
total proton counts, 0-6.

In [ ]:
slot_colors = [
    "white",
    "lightgray",
    "dodgerblue",
    "pink",
    "salmon",
    "greenyellow",
    "skyblue",
]

# The colors for the 19 slots, slot_color_list, according to
# the number of protons in each.
slot_color_list = np.zeros(19, dtype="object")
for i in range(19):
    slot_color_list[i] = slot_colors[slotlistA[i]]

# Defining the slot buttons by adding the slot position, count and colors
letters = [
    "Z",
    "Y",
    "Y",
    "Z",
    "Z",
    "Y",
    "Y",
    "X",
    "X",
    "Z",
    "Z",
    "Z",
    "Z",
    "Y",
    "Y",
    "Z",
    "Z",
    "Z",
    "Z",
]
slot_buttons = []

for i in range(19):
    slot_buttons.append(
        Button(
            description=f"{i+1} {letters[i]} {slotlistA[i]}",
            style=dict(button_color=slot_color_list[i]),
        )
    )

for i in range(7):
    slot_buttons.append(
        Button(description=str(i), style=dict(button_color=slot_colors[i]))
    )

for i in range(19):
    slot_buttons.append(
        Button(
            description=f"{i+1} {letters[i]}",
            style=dict(button_color=slot_color_list[i]),
        )
    )

# The slotlayout atom
slot_buttons.append(
    Button(
        description=atomic_symbols[selected_atomic_number - 1] + " " + str(selected_atomic_number),
        style=dict(button_color=slot_colors[0], font_weight="bold"),
    )
)

for button in slot_buttons:
    button.layout = widgets.Layout(border="1px solid black")
    button.layout.width = "100%"
    button.layout.height = "100%"

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
slot_grid = GridspecLayout(10, 9)

slot_grid[1, 4] = slot_buttons[13]  # Cr6-Elements.csv SlotLayout.
slot_grid[2, 4] = slot_buttons[3]
slot_grid[2, 5] = slot_buttons[8]
slot_grid[2, 6] = slot_buttons[12]
slot_grid[3, 3] = slot_buttons[17]
slot_grid[3, 4] = slot_buttons[1]
slot_grid[3, 5] = slot_buttons[15]
slot_grid[4, 2] = slot_buttons[9]
slot_grid[4, 3] = slot_buttons[5]
slot_grid[4, 4] = slot_buttons[0]
slot_grid[4, 5] = slot_buttons[6]
slot_grid[4, 6] = slot_buttons[11]
slot_grid[5, 3] = slot_buttons[16]
slot_grid[5, 4] = slot_buttons[2]
slot_grid[5, 5] = slot_buttons[18]
slot_grid[6, 2] = slot_buttons[10]
slot_grid[6, 3] = slot_buttons[7]
slot_grid[6, 4] = slot_buttons[4]
slot_grid[7, 4] = slot_buttons[14]
slot_grid[9, 1] = slot_buttons[19]  # Color/num protons legend
slot_grid[9, 2] = slot_buttons[20]  # along the bottom row
slot_grid[9, 3] = slot_buttons[21]
slot_grid[9, 4] = slot_buttons[22]
slot_grid[9, 5] = slot_buttons[23]
slot_grid[9, 6] = slot_buttons[24]
slot_grid[9, 7] = slot_buttons[25]
slot_grid[0, 0] = slot_buttons[26]  # L/R Vertical slot columns
slot_grid[0, 8] = slot_buttons[27]
slot_grid[1, 0] = slot_buttons[28]
slot_grid[1, 8] = slot_buttons[29]
slot_grid[2, 0] = slot_buttons[30]
slot_grid[2, 8] = slot_buttons[31]
slot_grid[3, 0] = slot_buttons[32]
slot_grid[3, 8] = slot_buttons[33]
slot_grid[4, 0] = slot_buttons[34]
slot_grid[4, 8] = slot_buttons[35]
slot_grid[5, 0] = slot_buttons[36]
slot_grid[5, 8] = slot_buttons[37]
slot_grid[6, 0] = slot_buttons[38]
slot_grid[6, 8] = slot_buttons[39]
slot_grid[7, 0] = slot_buttons[40]
slot_grid[7, 8] = slot_buttons[41]
slot_grid[8, 0] = slot_buttons[42]
slot_grid[8, 8] = slot_buttons[43]
slot_grid[9, 0] = slot_buttons[44]
slot_grid[9, 8] = slot_buttons[45]  # Atomic symbpl and number

The Slot Layout Diagram.

all atoms can be described by the number of protons (0-6) in each of the atom's (1-19) occupied slot positions.

The main vertical up/down channel slots (top to bottom) are (14,4,2,1,3,5,15). There is also the front/back channel (11,8,1,9,13), and left/right channel (10,6,1,7,12); or refer to the directions from center slot 1: +/-x, +/-y, and main vertical column +/-z. The front/back/ left/right arms spin as a single group, the Carousel, around slot 1. Hydroden has a single proton in its single occupied slot 1. Helium's single slot 1 contains two protons. Hook slot positions: 16,17,18, and 19, are joined to the main up/down column at slots, 2 and 3.

all but most of the bottom row buttons contain number pairs separated by either X, Y or Z, the direction of that slot's proton equatorial emission plane viewed edgewise, orthogonal to the proton's spin axis. The left side number indicates the 1-19 slot number. The right side number indicates the number of protons in that atom's slot.

The Slot Layout is shown in the atomic configuration in the center of the diagram. It is a 3D, six pointed star projected onto the diagram's flat surface. The slot button information is repeated in the left and right side vertical columns for convienence. The seven center buttons in the bottom row contains the total protons per slot color legend. The selected atom’s symbol and atomic number are in the bottom right corner.

In [ ]:
""" Create a SlotLayout Diagram including slot proton spin directions: 
R, L, A or B; and proton pole electron positions: T, B, or A. An 
'Electrons' value of A means electrons are in a TB configuration, 
at the top proton pole positions at the positive end of slot, and 
at the bottom pole positions at the slot's negative end. BT 
electron positions are not allowed.
A 'SlotSpin' value of A means the 2-directional slot proton at 
the top spins left, while the bottom proton spins right. A value 
of B means the opposite, the proton the top spins right while 
the bottom proton spins left. 
Occupied slots in the center SlotLayout contain 5 digits: 1. Electron 
position at the proton's T or B or A (both T and B). 2. The slot 
number. 3. The slot's proton emission plane orientation. 4. The 
color-coded number of protons in the slot. 5. The slot's R,L,A 
(LR) or B (RL) proton spin directions. This atomic model 
simplifies things in that each proton may be accompanied by both 
an electron and a neutron which spin about the proton poles as a 
single object. Each object should spin independently"""

# The 7 colors used for color-coding each slot's total proton counts, 0-6.
slot_proton_count_colors = [
    "white",
    "lightgray",
    "dodgerblue",
    "pink",
    "salmon",
    "greenyellow",
    "skyblue",
]

snatomic_symbols = slotlistA

# The colors for the 19 slots, slot_color_list, according to
# the number of protons in each.
slot_proton_count_color_list = np.zeros(19, dtype="object")
for i in range(19):
    slot_proton_count_color_list[i] = slot_proton_count_colors[slotlistA[i]]

# Defining the slot buttons by adding the slot position, count and colors.
# If the slot is occupied, include 5 digits, unoccupied slots show 3 digits.
slot_proton_number_buttons = []

descriptions = [
    "1Z",
    "2Y",
    "3Y",
    "4Z",
    "5Z",
    "6Y",
    "7Y",
    "8X",
    "9X",
    "AZ",
    "BZ",
    "CZ",
    "DZ",
    "EY",
    "FY",
    "GZ",
    "HZ",
    "IZ",
    "JZ",
]

for i in range(19):
    if slotsetA_df.loc[i + 1, "Protons"] == 0:
        slot_proton_number_buttons.append(
            Button(
                description=descriptions[i] + str(snatomic_symbols[i]),
                style=dict(button_color=slot_proton_count_color_list[i]),
            )
        )
    elif slotsetA_df.loc[i + 1, "Protons"] > 0:
        slot_proton_number_buttons.append(
            Button(
                description=str(slotsetA_df.loc[i + 1, "Electrons"])
                + descriptions[i]
                + str(snatomic_symbols[i])
                + str(slotsetA_df.loc[i + 1, "SlotSpin"]),
                style=dict(button_color=slot_proton_count_color_list[i]),
            )
        )

# Slotlayout proton total color legend
for i in range(7):
    slot_proton_number_buttons.append(
        Button(
            description=str(i),
            style=dict(button_color=slot_proton_count_colors[i]),
        )
    )
# Left and right slot columns,
# sn27 = Button(description='1 Z '+str(snatomic_symbols[0]), style=dict(button_color=slot_proton_count_color_list[0]))

for i in range(19):
    slot_proton_number_buttons.append(
        Button(
            description=descriptions[i] + str(snatomic_symbols[i]),
            style=dict(button_color=slot_proton_count_color_list[i]),
        )
    )

# The slotlayout atom
slot_proton_number_buttons.append(
    Button(
        description=atomic_symbols[selected_atomic_number - 1] + " " + str(selected_atomic_number),
        style=dict(button_color=slot_colors[0], font_weight="bold"),
    )
)

# Add each button's border, layout width and height.
for button in slot_proton_number_buttons:
    button.layout = widgets.Layout(border="1px solid black")
    button.layout.width = "100%"
    button.layout.height = "100%"

# Layout the 46 (19 + 19 + 7 + 1) slotlayout buttons
sngrid = GridspecLayout(10, 9)

sngrid[1, 4] = slot_proton_number_buttons[13]  # Cr6-Elements.csv SlotLayout.
sngrid[2, 4] = slot_proton_number_buttons[3]
sngrid[2, 5] = slot_proton_number_buttons[8]
sngrid[2, 6] = slot_proton_number_buttons[12]
sngrid[3, 3] = slot_proton_number_buttons[17]
sngrid[3, 4] = slot_proton_number_buttons[1]
sngrid[3, 5] = slot_proton_number_buttons[15]
sngrid[4, 2] = slot_proton_number_buttons[9]
sngrid[4, 3] = slot_proton_number_buttons[5]
sngrid[4, 4] = slot_proton_number_buttons[0]
sngrid[4, 5] = slot_proton_number_buttons[6]
sngrid[4, 6] = slot_proton_number_buttons[11]
sngrid[5, 3] = slot_proton_number_buttons[16]
sngrid[5, 4] = slot_proton_number_buttons[2]
sngrid[5, 5] = slot_proton_number_buttons[18]
sngrid[6, 2] = slot_proton_number_buttons[10]
sngrid[6, 3] = slot_proton_number_buttons[7]
sngrid[6, 4] = slot_proton_number_buttons[4]
sngrid[7, 4] = slot_proton_number_buttons[14]
sngrid[9, 1] = slot_proton_number_buttons[19]  # Color/num protons legend
sngrid[9, 2] = slot_proton_number_buttons[20]  # along the bottom row
sngrid[9, 3] = slot_proton_number_buttons[21]
sngrid[9, 4] = slot_proton_number_buttons[22]
sngrid[9, 5] = slot_proton_number_buttons[23]
sngrid[9, 6] = slot_proton_number_buttons[24]
sngrid[9, 7] = slot_proton_number_buttons[25]
sngrid[0, 0] = slot_proton_number_buttons[26]  # L/R Vertical slot columns
sngrid[0, 8] = slot_proton_number_buttons[27]
sngrid[1, 0] = slot_proton_number_buttons[28]
sngrid[1, 8] = slot_proton_number_buttons[29]
sngrid[2, 0] = slot_proton_number_buttons[30]
sngrid[2, 8] = slot_proton_number_buttons[31]
sngrid[3, 0] = slot_proton_number_buttons[32]
sngrid[3, 8] = slot_proton_number_buttons[33]
sngrid[4, 0] = slot_proton_number_buttons[34]
sngrid[4, 8] = slot_proton_number_buttons[35]
sngrid[5, 0] = slot_proton_number_buttons[36]
sngrid[5, 8] = slot_proton_number_buttons[37]
sngrid[6, 0] = slot_proton_number_buttons[38]
sngrid[6, 8] = slot_proton_number_buttons[39]
sngrid[7, 0] = slot_proton_number_buttons[40]
sngrid[7, 8] = slot_proton_number_buttons[41]
sngrid[8, 0] = slot_proton_number_buttons[42]
sngrid[8, 8] = slot_proton_number_buttons[43]
sngrid[9, 0] = slot_proton_number_buttons[44]
sngrid[9, 8] = slot_proton_number_buttons[45]  # Atomic symbol and number

In [ ]:
# Build each atom from Cr6-Elements

# atomicNumber = number
# Select the desired atom from the dropdown widget or table.
if 1 <= selected_atomic_number < 13:
    CAROUSEL_ROTATION_SECONDS = 2.5
if 14 <= selected_atomic_number < 20:
    CAROUSEL_ROTATION_SECONDS = 4.0
if 21 <= selected_atomic_number:
    CAROUSEL_ROTATION_SECONDS = 6.0

groupA1 = Group()  # Carousel. The corner rotation widget spins groupA1,
# slots 6,7,8,9,10,11,12,13.
groupA2 = Group()  # groupA2 contains slot1 and the remaining non-carousel
# orthogonal top, bottom and hook slots.
groupB1 = Group()  # Atom 2 Slot 1 and carousel
groupB2 = Group()  # Atom 2 remaining slots

# Each of the 1-19 slots of a given atom may have either right
# (A stacks) or left (B stacks) spinning proton groups. 38 Total.

stacks_A = [Group() for _ in range(19)]

stacks_B = [Group() for _ in range(19)]

""" 
Widget Controlled variables 1. global emR. EMISSION_RADIUS. The equatorial 
plane (or cone) emission radius displayed. 2. global s1s2. o-slots 
gap, the distance between orthogonal protons or orthogonal proton 
stacks. 3. Global p2p3Dist. epn gap. The distance between adjacent 
e-p-n proton groups. 4. global neutron_orbit_radius. The orbital offfset 
distance about the direct charge channel. 5. global c_angle_degree, the 
proton emission cone angle in degrees.
"""
# Dimensions # PROTON_RADIUS is now in cell In[1]
NEUTRON_RADIUS = PROTON_RADIUS * 1.125
ELECTRON_RADIUS = 2 * PROTON_RADIUS / 7.5
EMISSION_RADIUS = 2 * emission_radius * PROTON_RADIUS  # 2*pE - PROTON_RADIUS/2
CONE_ANGLE = c_angle_degree * 2 * PI_PHI / 360
CONE_HEIGHT = EMISSION_RADIUS * tan(CONE_ANGLE)
p12 = 2 * s1s2 * PROTON_RADIUS
pE = p12
p2p3 = 2 * p2p3Dist * PROTON_RADIUS
p1e1 = PROTON_RADIUS + 0.05 * p12  # proton-electron separation
n1n2 = 2 * NEUTRON_RADIUS * neutron_orbit_radius

# Proton set Geometries and Materials # Color.js Find a nice color spectrum
proton_geometry = SphereBufferGeometry(PROTON_RADIUS, 24, 16)
neutron_geometry = SphereBufferGeometry(NEUTRON_RADIUS, 24, 16)
electron_geometry = SphereBufferGeometry(ELECTRON_RADIUS, 8, 16)
emission_geometry_P = CircleGeometry(EMISSION_RADIUS, 24)
emission_geometry_X = ConeGeometry(EMISSION_RADIUS, CONE_HEIGHT, 16, openEnded=True)
proton_material = MeshStandardMaterial(color="blue")
neutron_material = MeshStandardMaterial(color="red")
electron_material = MeshStandardMaterial(
    color="cyan"
)  #'orangered','salmon','green','cyan','teal','orange'
emission_material7 = MeshStandardMaterial(
    color="gray", side="DoubleSide", transparent=True, opacity=0.20
)
emission_material1 = MeshStandardMaterial(
    color="magenta", side="DoubleSide", transparent=True, opacity=0.20
)
emission_material2 = MeshStandardMaterial(
    color="blue", side="DoubleSide", transparent=True, opacity=0.20
)
emission_material3 = MeshStandardMaterial(
    color="pink", side="DoubleSide", transparent=True, opacity=0.20
)
emission_material4 = MeshStandardMaterial(
    color="salmon", side="DoubleSide", transparent=True, opacity=0.20
)
emission_material5 = MeshStandardMaterial(
    color="lime", side="DoubleSide", transparent=True, opacity=0.20
)
emission_material6 = MeshStandardMaterial(
    color="skyblue", side="DoubleSide", transparent=True, opacity=0.20
)

emission_material_list = [
    emission_material1,
    emission_material2,
    emission_material3,
    emission_material4,
    emission_material5,
    emission_material6,
    emission_material7,
]

if emission_type.value == "plane":
    emission_geometry = emission_geometry_P
if emission_type.value == "cones":
    emission_geometry = emission_geometry_X


def buildProStackM(a, j):
    """
    Parameters: a - total number of slot protons. j - given slot #.

    buildProStackM(a,j) is an attempt to follow Miles Mathis'
    paper,'Diatomic Hydrogen' (DH), which includes electron-proton
    (e-p) to e-p charge bonding summarized in rules 1, 2, and 3:
    Rule 1. Two protons, each with any spin direction (L or R) can
    bond when they share a T B electron position configuration.
    Rule 2. T T or B B electron position configurations will be valid
    only if both protons have the same (L or R) spin direction.
    Rule 3. No combination of B T electrons are valid, two electrons
    between two protons prevent bonding.
    """

    if colored_emissions.value == True:
        for k in range(6):
            if a == k + 1:
                emission_material = emission_material_list[k]
    else:
        emission_material = emission_material_list[6]

    """ 
    #p2p3 = 2*p2p3Dist*PROTON_RADIUS, distance added for each additional e-p.
    The six sets of equidistant proton positions for a slot with 1-6 protons. 
    The proton (groups) are added beginning at the slot center. 5,3,1,2,4,6.
    """

    if a == 1:
        proton_positions = [0]
    elif a == 2:
        proton_positions = [p2p3 / 2, -p2p3 / 2]
    elif a == 3:
        proton_positions = [0, -p2p3, p2p3]
    elif a == 4:
        proton_positions = [p2p3 / 2, -p2p3 / 2, 3 * p2p3 / 2, -3 * p2p3 / 2]
    elif a == 5:
        proton_positions = [0, -p2p3, p2p3, -2 * p2p3, 2 * p2p3]
    elif a == 6:
        proton_positions = [
            p2p3 / 2,
            -p2p3 / 2,
            3 * p2p3 / 2,
            -3 * p2p3 / 2,
            5 * p2p3 / 2,
            -5 * p2p3 / 2,
        ]

    def cones_plus_plane(proton_mi, emission1, emission2, position):
        emission1.add(Mesh(emission_geometry, emission_material))
        # emission1.position = [0,0,CONE_HEIGHT/2]
        emission1.position = [0, 0, position]
        emission1.rotateX(-PI_PHI / 2)
        proton_mi.add(emission1)
        emission2.add(Mesh(emission_geometry, emission_material))
        # emission2.position = [0,0,-CONE_HEIGHT/2]
        emission2.position = [0, 0, -position]
        emission2.rotateX(PI_PHI / 2)
        proton_mi.add(emission2)
        proton_mi.add(Mesh(emission_geometry_P, emission_material))

    def add_electron(proton_mi, electron_mi, position):
        electron_mi.add(Mesh(electron_geometry, electron_material))
        electron_mi.position = [0, -n1n2 / 10, position]
        electron_mi.rotateZ(random() * 2 * PI_PHI)
        proton_mi.add(electron_mi)

    def single_neutron(proton_mi, neutron, position):
        neutron.add(Mesh(neutron_geometry, neutron_material))
        neutron.position = [0, n1n2 / 2, position]
        neutron.rotateZ(random() * 2 * PI_PHI)
        if display_neutrons is True:
            proton_mi.add(neutron)

    def pair_of_neutrons(proton_mi, neutron1, neutron2, position):
        neutron1.add(Mesh(neutron_geometry, neutron_material))
        neutron1.position = [0, -n1n2 / 2, 0]
        neutron2.add(Mesh(neutron_geometry, neutron_material))
        neutron2.position = [0, n1n2 / 2, 0]
        neutron_pair.add(neutron1)
        neutron_pair.add(neutron2)
        neutron_pair.rotateZ(random() * 2 * PI_PHI)
        neutron_pair.position = [0, 0, position]
        if display_neutrons is True:
            proton_mi.add(neutron_pair)

    # Increments the occupied slot's 1-6 proton groups.
    for i in range(a):
        proton_mi = Group()
        proton_mi.add(Mesh(proton_geometry, proton_material))
        if emission_type.value == "plane":
            proton_mi.add(Mesh(emission_geometry, emission_material))
        if emission_type.value == "cones":
            emission1 = Object3D()
            emission2 = Object3D()
            cones_plus_plane(proton_mi, emission1, emission2, CONE_HEIGHT / 2)

        # Electron and neutron additions to proton spin groups.
        electron_mi = Object3D()
        neutron_mi = Object3D()
        neutron_ni = Object3D()
        neutron_pair = Group()

        # Building the various T,B,A, and R,L,A slot configurations.
        intColumn = 8 + i  # Pointing to the N1-N6 data locations.
        intValue = slotsetA_df.iloc[j, intColumn]
        if slotsetA_df.loc[j + 1, ("SlotSpin")] == "R" or "L":
            if slotsetA_df.loc[j + 1, ("Electrons")] == "T":
                add_electron(proton_mi, electron_mi, p1e1)
                if intValue == 1:
                    single_neutron(proton_mi, neutron_mi, -p2p3 / 2)
                elif intValue == 2:
                    pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, -p2p3 / 2)
            if slotsetA_df.loc[j + 1, ("Electrons")] == "B":
                add_electron(proton_mi, electron_mi, -p1e1)
                if intValue == 1:
                    single_neutron(proton_mi, neutron_mi, p2p3 / 2)
                elif intValue == 2:
                    pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, p2p3 / 2)
            if slotsetA_df.loc[j + 1, ("Electrons")] == "A":  # 'SlotSpin' = R or L
                if i % 2 != 0:
                    add_electron(proton_mi, electron_mi, -p1e1)
                    if intValue == 1:
                        single_neutron(proton_mi, neutron_mi, p2p3 / 2)
                    elif intValue == 2:
                        pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, p2p3 / 2)
                elif i % 2 == 0:
                    add_electron(proton_mi, electron_mi, p1e1)
                    if intValue == 1:
                        single_neutron(proton_mi, neutron_mi, -p2p3 / 2)
                    elif intValue == 2:
                        pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, -p2p3 / 2)
        if (
            slotsetA_df.loc[j + 1, ("SlotSpin")] == "A"
            and slotsetA_df.loc[j + 1, ("Electrons")] == "A"
        ):
            if i == 0:
                add_electron(proton_mi, electron_mi, p1e1)
                if intValue == 1:
                    single_neutron(proton_mi, neutron_mi, p2p3 / 2)
            if i == 1:
                if intValue == 0:
                    add_electron(proton_mi, electron_mi, -p1e1)
                if intValue == 1:
                    add_electron(proton_mi, electron_mi, p1e1)
                    single_neutron(proton_mi, neutron_mi, -p2p3 / 2)
                if intValue == 2:
                    add_electron(proton_mi, electron_mi, p1e1)
                    pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, -p2p3 / 2)
                    proton_mi.rotateX(PI_PHI)
                    proton_mi.rotateZ(random() * 2 * PI_PHI)
            if i > 1 and i % 2 == 0:
                add_electron(proton_mi, electron_mi, p1e1)
                if intValue == 1:
                    single_neutron(proton_mi, neutron_mi, -p2p3 / 2)
                elif intValue == 2:
                    pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, -p2p3 / 2)
            if i > 1 and i % 2 != 0:
                if intValue == 1:
                    add_electron(proton_mi, electron_mi, -p1e1)
                    single_neutron(proton_mi, neutron_mi, p2p3 / 2)
                elif intValue == 2:
                    add_electron(proton_mi, electron_mi, -p1e1)
                    pair_of_neutrons(proton_mi, neutron_mi, neutron_ni, p2p3 / 2)

        proton_mi.rotateZ(random() * 2 * PI_PHI)
        proton_mi.position = [0, 0, proton_positions[i]]

        # Slot spin assignments
        if slotsetA_df.loc[j + 1, ("SlotSpin")] == "R":
            stacks_A[j].add(proton_mi)
        elif slotsetA_df.loc[j + 1, ("SlotSpin")] == "L":
            stacks_B[j].add(proton_mi)
        elif slotsetA_df.loc[j + 1, ("SlotSpin")] == "A":
            # Double spin slot spin assignments.
            if i % 2 != 0:
                stacks_A[j].add(proton_mi)
            else:
                stacks_B[j].add(proton_mi)
        elif slotsetA_df.loc[j + 1, ("SlotSpin")] == "B":
            # Double spin slot spin assignments.
            if i % 2 != 0:
                stacks_B[j].add(proton_mi)
            else:
                stacks_A[j].add(proton_mi)


def Orient(n, stack):
    # proStack emission planes are initially oriented in the z-plane.
    # Reorient the stack otherwise according to the "SlotOrien" column.
    if slotsetA_df.loc[n + 1, ("SlotOrien")] == "Y":
        stack.rotateX(-PI_PHI / 2)
    if slotsetA_df.loc[n + 1, ("SlotOrien")] == "X":
        stack.rotateX(-PI_PHI / 2)
        stack.rotateY(PI_PHI / 2)


def Position(n, stack):
    # The default values are just used as position markers, +/- x, y or z
    # with respect to (0,0,0). Use those directions but calculate the slot's
    # position magnitude based on the values of p2p3, p12 and pE.
    loc = abs(
        slotsetA_df.loc[n + 1, ("P2P3")] * p2p3
        + slotsetA_df.loc[n + 1, ("P12")] * p12
        + slotsetA_df.loc[n + 1, ("PE")] * pE
    )
    # Determine the direction of the stack according to the presence and sign
    # (+/-) of slotsetA_df's occupied (x,y,z) positions. Either a value or a
    #  zero, '0'. If there's a value, let it be given the position magnitude.
    px = slotsetA_df.loc[i + 1, ("PX")]  #
    py = slotsetA_df.loc[i + 1, ("PY")]
    pz = slotsetA_df.loc[i + 1, ("PZ")]
    if px != 0:
        px = copysign(1, px)
    if py != 0:
        py = copysign(1, py)
    if pz != 0:
        pz = copysign(1, pz)
    stack.position = [px * loc, py * loc, pz * loc]


# Assemble the atomic models.
for i in range(19):
    if slotsetA_df.loc[i + 1, ("Protons")] > 0:
        numPs = slotsetA_df.loc[i + 1, ("Protons")]
        buildProStackM(numPs, i)
        Orient(i, stacks_A[i])
        Orient(i, stacks_B[i])
        Position(i, stacks_A[i])
        Position(i, stacks_B[i])
        # groupA1: Carousal slots: 6,7,8,9,10,11,12,13
        anSlots = [5, 6, 7, 8, 9, 10, 11, 12]  # index#=slot-1
        if i in anSlots:
            groupA1.add(stacks_A[i])
            groupA1.add(stacks_B[i])
        else:
            groupA2.add(stacks_A[i])
            groupA2.add(stacks_B[i])

# print(An)  # (' 2 helium He ', 2)
# print(slotlistA)
# print('moleculeOrNot ,',moleculeOrNot.value,', di-atom ,',numberM)
# slotsetA_df.to_csv('out.csv')

# slotsetA_df

In [ ]:
# The rendered scene

if camera_orient == "horizontal":
    camera = camera_z_axis_horiz
else:
    camera = camera_z_axis_vert

if atoms_label.value == True:  # Still shows UserWarning
    scene = Scene(
        children=[
            groupA1,
            groupA2,
            groupB1,
            groupB2,
            camera,
            key_light,
            ambient_light,
            make_text(atomic_label, height=0.25, position=[-1, 1, selected_atomic_number * 0.025 + 1.5]),
        ]
    )
else:
    scene = Scene(
        children=[groupA1, groupA2, groupB1, groupB2, camera, key_light, ambient_light]
    )

controller = OrbitControls(controlling=camera, screenSpacePanning=True)
camera.mode = camera_type.value
renderer = Renderer(
    camera=camera,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=VIEW_WIDTH,
    height=VIEW_HEIGHT,
)


# initialising the scene
scene = Scene()

# Note. The UserWarning appears to be due to non JSON compliant infinite
# values used in the pythreejs orbital camera. See the related issue. JSON
# serialization of OrbitControls fails with jupyter_client 7.0.3,opened
# on 22 Sep 21, https://github.com/jupyter-widgets/pythreejs/issues/366

# 16 April, switched to a combined camera, allowing the user to switch views
# https://github.com/jupyter-widgets/pythreejs/blob/master/examples/Combined%20Camera.ipynb
# Still getting a User Warning.

In [ ]:
# Atom 1 spin animations. Defining the rotation widget controls for slots 1 - 19.
# Slot 1 (and the carousel).
a_track = NumberKeyframeTrack(
    name=".rotation[z]", times=[0, CAROUSEL_ROTATION_SECONDS], values=[0, 2 * PI_PHI]
)
# Slots 2-19.
b_track = NumberKeyframeTrack(name=".rotation[z]", times=[0, 2], values=[0, 2 * PI_PHI])

# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.
# Defining the rotation widget controls for slots 1 - 19.

# The remaining widgets rotate the individual slots.
spin_clips = [AnimationClip(tracks=[a_track]) for _ in range(19)]
spin_actions = [
    AnimationAction(AnimationMixer(stacks_A[i]), spin_clips[i], stacks_A[i])
    for i in range(19)
]

spinA20_track = a_track
spinA20_clip = AnimationClip(tracks=[spinA20_track])
spinA20_action = AnimationAction(AnimationMixer(groupA1), spinA20_clip, groupA1)
spin_actions.append(spinA20_action)

rgridA = GridspecLayout(8, 7)

rgridA[1, 3] = spin_actions[14]
rgridA[2, 3] = spin_actions[4]
rgridA[2, 4] = spin_actions[9]
rgridA[2, 5] = spin_actions[13]
rgridA[3, 2] = spin_actions[18]
rgridA[3, 3] = spin_actions[2]
rgridA[3, 4] = spin_actions[16]
rgridA[4, 1] = spin_actions[10]
rgridA[4, 2] = spin_actions[6]
rgridA[4, 3] = spin_actions[1]
rgridA[4, 4] = spin_actions[7]
rgridA[4, 5] = spin_actions[12]
rgridA[5, 2] = spin_actions[17]
rgridA[5, 3] = spin_actions[3]
rgridA[5, 4] = spin_actions[19]
rgridA[6, 1] = spin_actions[11]
rgridA[6, 2] = spin_actions[8]
rgridA[6, 3] = spin_actions[5]
rgridA[7, 3] = spin_actions[15]
rgridA[7, 5] = spinA20_action

In [ ]:
# Second set of spin animations. Defining the rotation widget controls for slots 1 - 19.
# Slot 1 (and the carousel).
a_track = NumberKeyframeTrack(
    name=".rotation[z]", times=[0, CAROUSEL_ROTATION_SECONDS], values=[0, -2 * PI_PHI]
)
# Slots 2-19.
b_track = NumberKeyframeTrack(
    name=".rotation[z]", times=[0, 2], values=[0, -2 * PI_PHI]
)

# Rotation controls for slot 1, which spins the atom's carousel-left/right, front/back slots.
# Defining the rotation widget controls for slots 1 - 19.
# The remaining widgets rotate the individual slots.

spin_clips = [AnimationClip(tracks=[b_track]) for _ in range(19)]
spin_actions = [
    AnimationAction(AnimationMixer(stacks_B[i]), spin_clips[i], stacks_B[i])
    for i in range(19)
]

spinB20_clip = AnimationClip(tracks=[a_track])
spinB20_action = AnimationAction(AnimationMixer(groupB1), spinB20_clip, groupB1)
spin_actions.append(spinB20_action)

# Layout the 19 rotation controls
rgridB = GridspecLayout(8, 7)

rgridB[1, 3] = spin_actions[14]
rgridB[2, 3] = spin_actions[4]
rgridB[2, 4] = spin_actions[9]
rgridB[2, 5] = spin_actions[13]
rgridB[3, 2] = spin_actions[18]
rgridB[3, 3] = spin_actions[2]
rgridB[3, 4] = spin_actions[16]
rgridB[4, 1] = spin_actions[10]
rgridB[4, 2] = spin_actions[6]
rgridB[4, 3] = spin_actions[1]
rgridB[4, 4] = spin_actions[7]
rgridB[4, 5] = spin_actions[12]
rgridB[5, 2] = spin_actions[17]
rgridB[5, 3] = spin_actions[3]
rgridB[5, 4] = spin_actions[19]
rgridB[6, 1] = spin_actions[11]
rgridB[6, 2] = spin_actions[8]
rgridB[6, 3] = spin_actions[5]
rgridB[7, 3] = spin_actions[15]
# rgridB[7, 5] = spinB20_action

In [ ]:
# The widget tab enclosure.

# Including a non-interactive periodic table showing the atom selected.

b_buttons = []
for i in range(90):
    button = Button(
        description=atomic_symbols[i], style=dict(button_color=color_list[i])
    )
    if i == atomic_number - 1:
        button = Button(
            description=atomic_symbols[atomic_number - 1],
            style=dict(button_color="white", font_weight="bold"),
        )
        button.layout = widgets.Layout(border="2px solid red")
    else:
        button.layout = widgets.Layout(border="1px solid black")
    button.layout.width = "100%"
    button.layout.height = "100%"  #'40px'
    b_buttons.append(button)

# Layout the 90 buttons in periodic table form
gridb = GridspecLayout(10, 18)

gridb[0, 0] = b_buttons[0]
gridb[0, 17] = b_buttons[1]
gridb[1, 0] = b_buttons[2]
gridb[1, 1] = b_buttons[3]
for p in range(12, 18):
    gridb[1, p] = b_buttons[p - 8]
gridb[2, 0] = b_buttons[10]
gridb[2, 1] = b_buttons[11]
for n in range(12, 18):
    gridb[2, n] = b_buttons[n]
m = 0
for m in range(18):
    gridb[3, m] = b_buttons[18 + m]
k = 0
for k in range(18):
    gridb[4, k] = b_buttons[36 + k]
gridb[5, 0] = b_buttons[54]
gridb[5, 1] = b_buttons[55]
# gridb[5,2] = the Lanthanide series hole
for i in range(3, 18):
    gridb[5, i] = b_buttons[68 + i]
gridb[6, 0] = b_buttons[86]
gridb[6, 1] = b_buttons[87]
# gridb[6,2] = the Actinide series hole

for j in range(2, 17):
    # Lanthanide series
    gridb[7, j] = b_buttons[54 + j]

# Actinide series
gridb[8, 2] = b_buttons[88]
gridb[8, 3] = b_buttons[89]

# The tab enclosure
tab1 = VBox(children=[renderer])  # 3D Atom
tab2 = VBox(children=[slot_grid])  # Slotlayout (SL) diagram
tab3 = VBox(children=[sngrid])  # SL w #'s of' left/right neutrons
tab4 = VBox(children=[gridb])  # Periodic table

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, atomic_label)
tab.set_title(1, atomic_symbols[selected_atomic_number - 1] + " Slotlayout (SL)")
tab.set_title(2, atomic_symbols[selected_atomic_number - 1] + " SL 1.TBA 5.RLAB")
tab.set_title(3, "Periodic Table")

# VBox(children=[tab]) # Display a tab enclosure with all outputs.
# Displayed between the top (R) and bottom (L) rotation controls.

In [ ]:
"""Rotation controls for all slots' right spinning proton groups.
The Carousel's R spin control is at the bottom right. 
Less than half the slots conbtain only right spin groups and about as many 
other slots contain left spin groups. Some slots, i.e. slot1, contain both"""
rgridA

In [ ]:
VBox(children=[tab])  # Display a tab enclosure with all outputs

In [ ]:
rgridB  # Rotation controls for just the left spinning proton groups in all slots.


Charge Field physics and the Unified Field theory, (and much more) have been well developed and well described by Miles Mathis. See,

***THE GREATEST STANDING ERRORS IN PHYSICS AND MATHEMATICS***
<http://milesmathis.com/index.html>
[Miles Mathis science site Homepage](http://milesmathis.com/index.html).

**SECTION 9: THE NUCLEUS** contains descriptions and diagrams 
of charge channeling and charge recycling by the 
elements. A paper most relevant to Atom Builder is 
>**How to Build the Elements**. 
>Explaining the periodic table, with nuclear diagrams".  
<http://milesmathis.com/nuclear.pdf>
[How to Build the Elements](http://milesmathis.com/nuclear.pdf)
For Charge Field discussion visit the forum at "Miles Mathis' Charge Field - Portal" https://milesmathis.forumotion.com.

This particular project is described in the Miles Periodic Table with Standard Periodic Table reference thread, https://milesmathis.forumotion.com/t634p75-miles-periodic-table-with-standard-periodic-table-reference#6702.

This project was Cr6's idea. He's also responsible for following Miles Mathis' atomic model and creating the Slotlayout diagram this project greatly relies.

There's at least one 10 year old charge field "Atom Viewer" out there, Nevyn's object oriented, way sophisticated javaScript code. I'll try to figure it out.

Please pardon my understandings, personal interpretations and mistakes. Feel free to make this project better.